## Setup

In this notebook we solve a **steady, incompressible Navier–Stokes** problem coupled with a **thermal energy transport** equation.  
The coupled governing equations can be expressed as:

### Governing equations  

$$
\rho\,\mathbf{u}\,\cdot\nabla\mathbf{u}
-\nabla\!\cdot\!\Bigl[\mu\bigl(\nabla\mathbf{u}+\nabla\mathbf{u}^{\mathrm T}\bigr)\Bigr]
+\nabla p
+\alpha\,\mathbf{u}
= \mathbf{0}
\qquad\text{in } \Omega,
$$  

$$
\nabla\!\cdot\!\mathbf{u}=0
\qquad\text{in } \Omega.
$$

$$\mathbf{u} \cdot \nabla T - \nabla \cdot \left(\frac{1}{\text{Pe}} \nabla T\right) = 0 \quad \text{in } \Omega$$  

* $\mathbf u=(u_x,u_y)$ velocity  
* $p$ pressure  
* $\rho$ density  
* $\mu$ dynamic viscosity  
* $\alpha$ Brinkman penalisation that damps flow in solid zones.
* Peclet number Pe expressed as:
$$
  \mathrm{Pe}=\frac{U\,L}{\kappa},
  \qquad
  \kappa=\frac{k}{\rho\,c_p}
  $$
  where $U$ and $L$ are characteristic velocity and length, $\kappa$ is thermal diffusivity, $k$ is thermal conductivity, and $c_p$ is specific heat at constant pressure.

### Boundary conditions  

$$
\begin{aligned}
\mathbf{u} &= \mathbf{u}_{\text{in}}
&&\text{on } \Gamma_{\text{in}}\quad\text{(prescribed inflow)},\\[4pt]
-p\,\mathbf{n} + \mu\bigl(\nabla\mathbf{u}+\nabla\mathbf{u}^{\mathrm T}\bigr)\mathbf{n} &= \mathbf{0}
&&\text{on } \Gamma_{\text{out}}\quad\text{(zero-normal-stress outflow)},\\[4pt]
\mathbf{u} &= \mathbf{0}
&&\text{on } \Gamma_{\text{wall}}\quad\text{(no-slip walls)}.\\[4pt]
T &= T_{\text{in}}
&&\text{on } \Gamma_{\text{in}}\quad\text{(prescribed inflow temperature)},\\[4pt]
T &= T_{\text{wall}}
&&\text{on } \Gamma_{\text{wall}}\quad\text{(prescribed wall temperature)},\\[4pt]
\nabla T \cdot \mathbf{n} &= 0  
&&\text{on } \Gamma_{\text{out}}\quad\text{(zero diffusive flux / convective outflow)},
\end{aligned}
$$  

---

### Test case: Conjugate Heat Transfer (CHT) benchmark — heated channel 

<img src="../../figures/heat_channel.png" alt="Channel" style="width: 500px;"/>

A fully developed laminar flow enters a straight 2-D channel with prescribed
inlet temperature and exits through a zero temperature gradient normal to the outlet.
Top and bottom walls are heated.

- **Flow field:** obtained by solving the steady incompressible Navier–Stokes equations with Brinkman penalization; the velocity is then held fixed for the CHT solve (one-way coupling).
- **Thermal model:** advection–diffusion in the fluid and pure diffusion in the solid, coupled via interface continuity of temperature and heat flux.
- **Discretization:** bilinear quadrilateral elements (\(Q_1\)) for temperature over both subdomains; with SUPG stabilization applied.



In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt

import toflux.src.utils as _utils
import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc

import toflux.src.fe_thermal as _fea_thermal
import toflux.src.fe_fluid as _fea_fluid
import toflux.src.solver as _solv
import toflux.src.viz as _viz

_Ext = _utils.Extent


_FluidField = _fea_fluid.FluidField
_TempField = _fea_thermal.ThermalField

jax.config.update("jax_enable_x64", True)

## Define Geometry and Mesh

### Mesh

Below we build the **conjugate heat transfer (CHT)** channel benchmark.

#### Geometry  
The channel geometry (defined in JSON) is passed to the both meshers (fluid and thermal).

#### Discretisation  
* The domain is partitioned into **bilinear quadrilateral (Q1) elements** with same numbe rof elements for both fluid and CHT solver.  

#### Numerical integration  
Element contributions are evaluated with **$2\times2$** Gauss quadrature:

This mesh serves as the baseline for the finite-element **CHT** solve (advection–diffusion fluid region, diffusion in solid region) and for any subsequent **density-based topology-optimisation** runs involving fluid and thermal objectives.


In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/channel_flow.json")

nelx_desired, nely_desired, gauss_order = 200, 40, 2

thermal_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=nelx_desired,
  nely_desired=nely_desired,
  dofs_per_node=1,
  gauss_order=gauss_order,
)

fluid_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=nelx_desired,
  nely_desired=nely_desired,
  dofs_per_node=3,
  gauss_order=gauss_order,
)

_viz.plot_brep(geom)

## Material


For this CHT setup we specify **fluid** properties and **thermal** properties separately.  

#### Fluid (momentum)
| Symbol | Meaning | Value |
|---|---|---|
| $ \rho $ | Mass density | 1.2 |
| $ \mu $ | Dynamic viscosity | $1.8\times10^{-5}$ |

- **Brinkman term (optional):** $\alpha=\mu/K$. For this demo we set $\alpha=0$ (i.e., $K\to\infty$) to recover pure Navier–Stokes; in topology optimization, $\alpha$ is obtained via a RAMP map from the design variable $\gamma\in[0,1]$ to damp flow in solid-like regions.

#### Thermal (energy)
| Symbol | Meaning | Value |
|---|---|---|
| $ k $ | Thermal conductivity | 0.025 |
| $ c_p $ | Specific heat (constant pressure) | 1006|
| $ \rho $ | Mass density (thermal) | 1.2 |


For the nondimensional energy equation, the Péclet number is $ \mathrm{Pe}=\dfrac{U\,L}{\kappa} $ for chosen $U$ and $L$.

> With these definitions, the flow solver uses `fluid_mat` for momentum equations, and the CHT solver uses `thermal_mat` to assemble the advection–diffusion operator in the fluid and diffusion in the solid. The same mesh coordinates are shared, so $T$, $p$, and $\mathbf u$ are collocated at the nodes.


In [ ]:
fluid_mat = _mat.FluidMaterial(
  mass_density=1.2,
  dynamic_viscosity=1.8e-5,
)

thermal_mat = _mat.ThermalMaterial(
  thermal_conductivity=0.025,
  specific_heat=1006.0,
  mass_density=1.2,
)

## Fluid Boundray Conditions



| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(u = U_\text{c})$, $(v = 0)$ |
| **Outlet (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a channel Reynolds number of 100:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 100,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{channel height}.
$$


In [ ]:
reynolds_num = 100.0
characteristic_length = fluid_mesh.bounding_box.ly
char_velocity = reynolds_num * fluid_mat.kinematic_viscosity / characteristic_length


# inlet condition
inlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[3]])
n = len(inlet_faces)
u_vel = (_FluidField.U_VEL, char_velocity * jnp.ones(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[1]])
n = len(outlet_faces)
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
pres = (_FluidField.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# wall condition
wall_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[0], geom.edges[2]])
n = len(wall_faces)
u_vel = (_FluidField.U_VEL, jnp.zeros(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]


inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="In")
outlet_bc = _bc.DirichletBC(elem_faces=outlet_faces, values=outlet_face_val, name="Out")
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="Wall")

fluid_bc_list = [wall_bc, inlet_bc, outlet_bc]
fluid_bc = _bc.process_boundary_conditions(fluid_bc_list, fluid_mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_bc(fluid_bc_list, fluid_mesh, ax=ax)
plt.show()

## Thermal boundary condition



| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(T = 20)$ |
| **Outlet (right vertical edge)** | Neumann | $(\nabla T \cdot \mathbf{n} = 0)$ |
| **Top wall** | Dirichlet | $(T = 100)$ |
| **Bottom wall** | Dirichlet | $(T = 100)$ |


In [ ]:
temp_inlet = 20.0
temp_wall = 100.0
temp_ext = _Ext(min=temp_inlet, max=temp_wall)
temp_ext.normalize_array(20.)
# inlet condition
inlet_faces = _bc.identify_faces(thermal_mesh, edges=[geom.edges[3]])
n = len(inlet_faces)
tv = (_TempField.TEMPERATURE, temp_ext.normalize_array(20.) * jnp.ones(n))
inlet_face_val = [tv]

# wall condition
wall_faces = _bc.identify_faces(thermal_mesh, edges=[geom.edges[0], geom.edges[2]])
n = len(wall_faces)
tv = (_TempField.TEMPERATURE, temp_ext.normalize_array(100.)* jnp.ones(n))
wall_face_val = [tv]


inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="In")
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="Wall")

thermal_bc_list = [wall_bc, inlet_bc]
thermal_bc = _bc.process_boundary_conditions(thermal_bc_list, thermal_mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
_viz.plot_bc(thermal_bc_list, thermal_mesh, ax=ax)
ax.set_title("Thermal Boundary Conditions")
plt.show()

# Solver

The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'. The energy equations are linear in temperature.  By default, all our FEA solvers are programmed as a subclass of nonlinear problems. Hence, the solver expects the settings for both the linear and nonlinear solvers.
Even when the physics is linear, the solver still performs a Newton–Raphson step; for a linear model this step converges in one iteration. 


In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.PETSC,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-4},
}

flow_solver = _fea_fluid.FluidSolver(
  mesh=fluid_mesh, bc=fluid_bc, material=fluid_mat, solver_settings=solver_settings
)


thermal_solver = _fea_thermal.FEA(
  mesh=thermal_mesh,
  material=thermal_mat,
  bc=thermal_bc,
  solver_settings=solver_settings,
)

# Solve
We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
press_vel = jnp.zeros((fluid_mesh.num_dofs,))
press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])

In [ ]:
density = np.zeros((fluid_mesh.num_elems,))
inv_permeability_ext = _utils.Extent(min=0.0, max=122.5)
brinkman_penalty = _mat.compute_ramp_interpolation(
  prop=density,
  ramp_penalty=8.0,
  prop_ext=inv_permeability_ext,
  mode="convex"
)

In [ ]:
press_vel = _solv.modified_newton_raphson_solve(
  flow_solver, press_vel, brinkman_penalty
)

## Plot the fields

We visualize the velocity  and pressure fields here.

In [ ]:
press_vel_elem = press_vel[fluid_mesh.elem_dof_mat]
press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
titls = ["Pressure", "U velocity", "V velocity", "Velocity magnitude"]

fig, ax = plt.subplots(2, 2)

ax = ax.flatten()
for i, (title, data) in enumerate(zip(titls, 
                                      [press_elem, u_vel_elem, v_vel_elem, 
                                       vel_elem_mag])):
  _viz.plot_grid_mesh(
    fluid_mesh,
    data.reshape(-1),
    ax[i])
  ax[i].set_title(title)


## Get the elemental velocities

Extract u and v  velocities from press_vel_elem and form an elemental velocity vector using mesh node ordering, then normalize by char_velocity to obtain conv_velocity.


In [ ]:
num_vel_dofs_per_elem = fluid_mesh.num_dim * fluid_mesh.elem_template.num_nodes
elem_vel = jnp.zeros((fluid_mesh.num_elems, num_vel_dofs_per_elem))
elem_vel = elem_vel.at[:, 0 :: fluid_mesh.num_dim].set(press_vel_elem[:, 1::3])
elem_vel = elem_vel.at[:, 1 :: fluid_mesh.num_dim].set(press_vel_elem[:, 2::3])
conv_velocity = elem_vel/char_velocity

## Solve the energy equation


In [ ]:
eps = 1e-12
temp = jnp.zeros((thermal_mesh.num_dofs,))
temp = temp.at[thermal_bc["fixed_dofs"]].set(thermal_bc["dirichlet_values"])


diffusivity_ext = _utils.Extent(
  min=eps, max=thermal_mat.diffusivity
)

eff_diffusivity = _mat.compute_ramp_interpolation(
  prop=np.ones((fluid_mesh.num_elems,)),
  ramp_penalty=0.01,
  prop_ext=diffusivity_ext,
  mode="concave",
)
peclet_number = characteristic_length * char_velocity / eff_diffusivity
tempr_non_dimensional = _solv.modified_newton_raphson_solve(
  thermal_solver, temp, conv_velocity, peclet_number
)
tempr = temp_ext.renormalize_array(tempr_non_dimensional)

## Visualize temperature

In [ ]:
elem_temp = tempr[thermal_mesh.elem_dof_mat].mean(axis=1)

fig, ax = plt.subplots()
_viz.plot_grid_mesh(
    thermal_mesh,
    elem_temp,
    ax)